In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import os.path
from pathlib import Path
import glob
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, SpatialDropout2D
from tensorflow.keras import layers
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
from tensorflow.keras.preprocessing import image
from PIL import Image
import psutil
print(f'current memory usage {psutil.virtual_memory().percent}%')

current memory usage 7.4%


In [ ]:
# Selecting folder paths in training_set
train_filepaths = []

for path, subdirs, files in os.walk('/home/jupyter/RealCNN/dataset/training_set/'):
    for name in files:
        train_filepaths.append(os.path.join(path, name))

# Mapping labels...
labels_training = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], train_filepaths))

# Training paths & labels
train_filepaths = pd.Series(train_filepaths, name = 'File').astype(str)
train_labels = pd.Series(labels_training, name='Label')

# Concatenating...
trainset_df = pd.concat([train_filepaths, train_labels], axis=1)
trainset_df.shape

In [ ]:
# Selecting folder paths in test_set
test_filepaths = []

for path, subdirs, files in os.walk('/home/jupyter/RealCNN/dataset/test_set/'):
    for name in files:
        test_filepaths.append(os.path.join(path, name))

# Mapping labels...
labels_test = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], test_filepaths))

# Test paths & labels
test_filepaths = pd.Series(test_filepaths, name = 'File').astype(str)
test_labels = pd.Series(labels_test, name='Label')

# Concatenating...
testset_df = pd.concat([test_filepaths, test_labels], axis=1)

testset_df.shape

In [ ]:
# Viewing data in both datasets
print('Training Dataset:')

print(f'Number of images in the training dataset: {trainset_df.shape[0]}')

print(f'Number of images with cats: {trainset_df["Label"].value_counts()[0]}')
print(f'Number of images with dogs: {trainset_df["Label"].value_counts()[1]}\n')
      
print('Test Dataset:')
      
print(f'Number of images in the test dataset: {testset_df.shape[0]}')

print(f'Number of images with cats: {testset_df["Label"].value_counts()[0]}')
print(f'Number of images with dogs: {testset_df["Label"].value_counts()[1]}')

In [ ]:
target_size = (96, 96, 3)
batch_size = 128
num_batches_per_epoch = 3

In [ ]:
# shuffle the data
trainset_df = trainset_df.sample(frac = 1, random_state = 56).reset_index(drop = True).head(num_batches_per_epoch*batch_size)
testset_df = testset_df.sample(frac = 1, random_state = 56).reset_index(drop = True)

display(trainset_df.head())

testset_df.head()

## 4. Generating batches of images
In this part we will generate batches of images increasing the training data, for the test database we will just normalize the data using [ImageDataGenerator](https://keras.io/api/preprocessing/image/#imagedatagenerator-class)

Parameters of ``ImageDataGenerator``:

    rescale - Transform image size (normalization of data)
    shear_range - Random geometric transformations
    zoom_range - Images that will be zoomed
    rotation_range - Degree of image rotation
    width_shift_range - Image Width Change Range
    height_shift_range - Image height change range
    horizontal_flip - Rotate images horizontally
    vertical_flip - Rotate images vertically
    validation_split - Images that have been reserved for validation (0-1)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.1,
                                    rotation_range = 20,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    horizontal_flip = True,
                                    vertical_flip = True,
                                    validation_split = 0.1)

test_datagen = ImageDataGenerator(rescale = 1./255)

## 5. Directory of training, validation and test images

Here we make the division of the image bases for training, validation and testing of the model, for that we use the [flow_from_dataframe](https://keras.io/api/preprocessing/image/#flowfromdataframe-method)

Parameters of ``flow_from_directory``:

    dataframe - Dataframe containing the images directory
    x_col - Column name containing the images directory
    y_col - Name of the column containing what we want to predict
    target_size - size of the images (remembering that it must be the same size as the input layer)
    color_mode - RGB color standard
    class_mode - binary class mode (cat/dog)
    batch_size - batch size (32)
    shuffle - Shuffle the data
    seed - optional random seed for the shuffle
    subset - Subset of data being training and validation (only used if using validation_split in ImageDataGenerator)

In [ ]:
trainset_df

In [ ]:

print("Preparing the training dataset ...")
training_set = train_datagen.flow_from_dataframe(
    dataframe = trainset_df,
    x_col = "File",
    y_col = "Label",
    target_size = target_size[:-1],
    color_mode = "rgb",
    class_mode = "binary",
    batch_size = batch_size,
    shuffle = True,
    seed = 2,
    subset = "training")

print("Preparing the validation dataset ...")
validation_set = train_datagen.flow_from_dataframe(
    dataframe = trainset_df,
    x_col = "File",
    y_col = "Label",
    target_size = target_size[:-1],
    color_mode ="rgb",
    class_mode = "binary",
    batch_size = batch_size,
    shuffle = True,
    seed = 2,
    subset = "validation")

print("Preparing the test dataset ...")
test_set = test_datagen.flow_from_dataframe(
    dataframe = testset_df,
    x_col = "File",
    y_col = "Label",
    target_size = target_size[:-1],
    color_mode ="rgb",
    class_mode = "binary",
    shuffle = False,
    batch_size = batch_size)

print('Data generators are ready!')

In [ ]:
def sample_from_generator(gen, nb_sample):
    from tqdm import tqdm
    cur_x, cur_y = next(gen)
    input_shape = list(cur_x.shape)[1:]
    batch_size = len(cur_x)

    X_sample = np.zeros([nb_sample] + list(input_shape))
    Y_sample = np.zeros((nb_sample,))
    
    
    X_sample[0:0 + batch_size] = cur_x
    Y_sample[0:0 + batch_size] = cur_y

    for i in tqdm(range(batch_size, nb_sample, batch_size)):
        cur_x, cur_y = next(gen)
        if len(X_sample[i:i + batch_size]) < len(cur_x):
            cur_x = cur_x[:len(X_sample[i:i + batch_size])]
            cur_y = cur_y[:len(Y_sample[i:i + batch_size])]
        if len(X_sample[i:i + batch_size]) >= len(cur_x):
            shape = cur_x.shape[0]
            X_sample[i:i + shape] = cur_x
            Y_sample[i:i + shape] = cur_y
    return X_sample, Y_sample

In [ ]:
X_sample, Y_sample = sample_from_generator(training_set, batch_size*num_batches_per_epoch)
X_sample.shape

In [ ]:

print("Preparing the training dataset ...")
training_set = train_datagen.flow_from_dataframe(
    dataframe = trainset_df,
    x_col = "File",
    y_col = "Label",
    target_size = target_size[:-1],
    color_mode = "rgb",
    class_mode = "binary",
    batch_size = batch_size,
    shuffle = True,
    seed = 2,
    subset = "training")

In [ ]:
def Alex_model(compile_model=False):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, SpatialDropout2D
    #Instantiation
    AlexNet = Sequential()

    #1st Convolutional Layer
    AlexNet.add(Conv2D(filters=96, input_shape=target_size, kernel_size=(11,11), strides=(4,4), padding='same', activation='relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #2nd Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same', activation='relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #3rd Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

    #4th Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

    #5th Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #Passing it to a Fully Connected layer
    AlexNet.add(Flatten())
    # 1st Fully Connected Layer
    AlexNet.add(Dense(4096, input_shape=(32,32,3,), activation='relu'))
    # Add Dropout to prevent overfitting
    AlexNet.add(Dropout(0.4))

    #2nd Fully Connected Layer
    AlexNet.add(Dense(4096, activation='relu'))
    #Add Dropout
    AlexNet.add(Dropout(0.4))

    #3rd Fully Connected Layer
    AlexNet.add(Dense(1000, activation='relu'))
    #Add Dropout
    AlexNet.add(Dropout(0.4))

    #Output Layer
    AlexNet.add(Dense(1, activation='softmax'))
    
    if compile_model:
        AlexNet.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return AlexNet

    

In [ ]:
# Callbacks
from model_trainingtime_prediction.model_level_utils_cnn import convert_cnn2d_data, TimeHistory

time_callback_data = TimeHistory()
time_callback_gen_data = TimeHistory()

CNN_data = Alex_model(compile_model=True)
CNN_gen_data = Alex_model(compile_model=True)

# Train
CNN_model = CNN_data.fit(X_sample,Y_sample, batch_size = batch_size, epochs = 3, callbacks = [time_callback_data], verbose=False)
CNN_model = CNN_gen_data.fit(training_set, batch_size = batch_size, epochs = 3, callbacks = [time_callback_gen_data], verbose=False)


In [ ]:
batch_times = np.array(time_callback_data.batch_times) * 1000
np.median(batch_times[1:])

In [ ]:
batch_times = np.array(time_callback_gen_data.batch_times) * 1000
np.median(batch_times[1:])

In [ ]:
mark_train_start = False
truncate_from = 1


In [ ]:
target_call_back = time_callback_data 
#target_call_back = time_callback_gen_data

target_call_back.relative_by_train_start()

epoch_data = []
for index,(start,end) in enumerate(target_call_back.epoch_times_detail[truncate_from:]):
    epoch_data.append((f'epoch{index+1+truncate_from} start', start))
    epoch_data.append((f'epoch{index+1+truncate_from} end', end))
batch_data = []
for index,(start,end) in enumerate(target_call_back.batch_times_detail[truncate_from*num_batches_per_epoch:]):
    batch_data.append((f'batch{index+1+truncate_from*num_batches_per_epoch} start', start))
    batch_data.append((f'batch{index+1+truncate_from*num_batches_per_epoch} end', end))
time_data = []
time_data.extend(epoch_data)
time_data.extend(batch_data)
if mark_train_start:
    time_data.append(('train start', 0))
time_data.append(('train end', target_call_back.train_end_time))
time_data = sorted(time_data, key=lambda x: x[1])

names = [i for (i,j) in time_data]
timestamp = [j for (i,j) in time_data]
# Choose some nice levels
levels = np.tile([-5, 5, -3, 3, -1, 1],
                 int(np.ceil(len(timestamp)/6)))[:len(timestamp)]

# Create figure and plot a stem plot with the date
fig, ax = plt.subplots(figsize=(25, 8), constrained_layout=True)
# ax.set(title="Model train timeline")

ax.vlines(timestamp, 0, levels, color="tab:red")  # The vertical stems.
ax.plot(timestamp, np.zeros_like(timestamp), "-o",
        color="k", markerfacecolor="w")  # Baseline and markers on it.

# annotate lines
for d, l, r in zip(timestamp, levels, names):
    ax.annotate(r, xy=(d, l),
                xytext=(-3, np.sign(l)*3), textcoords="offset points",
                horizontalalignment="right",
                verticalalignment="bottom" if l > 0 else "top", rotation=90, fontsize=20 if r.startswith('b') else 30)

plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

# remove y axis and spines
ax.get_yaxis().set_visible(False)
for spine in ["left", "top", "right"]:
    ax.spines[spine].set_visible(False)

ax.margins(y=0.1)
plt.show()


In [ ]:
#target_call_back = time_callback_data 
target_call_back = time_callback_gen_data

target_call_back.relative_by_train_start()

epoch_data = []
for index,(start,end) in enumerate(target_call_back.epoch_times_detail[truncate_from:]):
    epoch_data.append((f'epoch{index+1+truncate_from} start', start))
    epoch_data.append((f'epoch{index+1+truncate_from} end', end))
batch_data = []
for index,(start,end) in enumerate(target_call_back.batch_times_detail[truncate_from*num_batches_per_epoch:]):
    batch_data.append((f'batch{index+1+truncate_from*num_batches_per_epoch} start', start))
    batch_data.append((f'batch{index+1+truncate_from*num_batches_per_epoch} end', end))
time_data = []
time_data.extend(epoch_data)
time_data.extend(batch_data)
if mark_train_start:
    time_data.append(('train start', 0))
time_data.append(('train end', target_call_back.train_end_time))
time_data = sorted(time_data, key=lambda x: x[1])

names = [i for (i,j) in time_data]
timestamp = [j for (i,j) in time_data]
# Choose some nice levels
levels = np.tile([-5, 5, -3, 3, -1, 1],
                 int(np.ceil(len(timestamp)/6)))[:len(timestamp)]

# Create figure and plot a stem plot with the date
fig, ax = plt.subplots(figsize=(25, 8), constrained_layout=True)
# ax.set(title="Model train timeline")

ax.vlines(timestamp, 0, levels, color="tab:red")  # The vertical stems.
ax.plot(timestamp, np.zeros_like(timestamp), "-o",
        color="k", markerfacecolor="w")  # Baseline and markers on it.

# annotate lines
for d, l, r in zip(timestamp, levels, names):
    ax.annotate(r, xy=(d, l),
                xytext=(-3, np.sign(l)*3), textcoords="offset points",
                horizontalalignment="right",
                verticalalignment="bottom" if l > 0 else "top", rotation=90, fontsize=20 if r.startswith('b') else 30)

plt.setp(ax.get_xticklabels(), rotation=30, ha="right")

# remove y axis and spines
ax.get_yaxis().set_visible(False)
for spine in ["left", "top", "right"]:
    ax.spines[spine].set_visible(False)

ax.margins(y=0.1)
plt.show()


# if data gen is parallel

In [2]:
# Selecting folder paths in training_set
train_filepaths = []

for path, subdirs, files in os.walk('/home/jupyter/RealCNN/dataset/training_set/'):
    for name in files:
        train_filepaths.append(os.path.join(path, name))

# Mapping labels...
labels_training = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], train_filepaths))

# Training paths & labels
train_filepaths = pd.Series(train_filepaths, name = 'File').astype(str)
train_labels = pd.Series(labels_training, name='Label')

# Concatenating...
trainset_df = pd.concat([train_filepaths, train_labels], axis=1)
trainset_df.shape

(8000, 2)

In [3]:
# Selecting folder paths in test_set
test_filepaths = []

for path, subdirs, files in os.walk('/home/jupyter/RealCNN/dataset/test_set/'):
    for name in files:
        test_filepaths.append(os.path.join(path, name))

# Mapping labels...
labels_test = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], test_filepaths))

# Test paths & labels
test_filepaths = pd.Series(test_filepaths, name = 'File').astype(str)
test_labels = pd.Series(labels_test, name='Label')

# Concatenating...
testset_df = pd.concat([test_filepaths, test_labels], axis=1)

testset_df.shape

(2000, 2)

In [4]:
# Viewing data in both datasets
print('Training Dataset:')

print(f'Number of images in the training dataset: {trainset_df.shape[0]}')

print(f'Number of images with cats: {trainset_df["Label"].value_counts()[0]}')
print(f'Number of images with dogs: {trainset_df["Label"].value_counts()[1]}\n')
      
print('Test Dataset:')
      
print(f'Number of images in the test dataset: {testset_df.shape[0]}')

print(f'Number of images with cats: {testset_df["Label"].value_counts()[0]}')
print(f'Number of images with dogs: {testset_df["Label"].value_counts()[1]}')

Training Dataset:
Number of images in the training dataset: 8000
Number of images with cats: 4000
Number of images with dogs: 4000

Test Dataset:
Number of images in the test dataset: 2000
Number of images with cats: 1000
Number of images with dogs: 1000


In [5]:
target_size = (128, 128, 3)
batch_size = 128
num_batches_per_epoch = 60

In [6]:
# shuffle the data
trainset_df = trainset_df.sample(frac = 1, random_state = 56).reset_index(drop = True).head(num_batches_per_epoch*batch_size)
testset_df = testset_df.sample(frac = 1, random_state = 56).reset_index(drop = True)

display(trainset_df.head())

testset_df.head()

,File,Label
0,/home/jupyter/RealCNN/dataset/training_set/cat...,cats
1,/home/jupyter/RealCNN/dataset/training_set/cat...,cats
2,/home/jupyter/RealCNN/dataset/training_set/dog...,dogs
3,/home/jupyter/RealCNN/dataset/training_set/cat...,cats
4,/home/jupyter/RealCNN/dataset/training_set/cat...,cats


,File,Label
0,/home/jupyter/RealCNN/dataset/test_set/cats/ca...,cats
1,/home/jupyter/RealCNN/dataset/test_set/cats/ca...,cats
2,/home/jupyter/RealCNN/dataset/test_set/cats/ca...,cats
3,/home/jupyter/RealCNN/dataset/test_set/dogs/do...,dogs
4,/home/jupyter/RealCNN/dataset/test_set/dogs/do...,dogs


## 4. Generating batches of images
In this part we will generate batches of images increasing the training data, for the test database we will just normalize the data using [ImageDataGenerator](https://keras.io/api/preprocessing/image/#imagedatagenerator-class)

Parameters of ``ImageDataGenerator``:

    rescale - Transform image size (normalization of data)
    shear_range - Random geometric transformations
    zoom_range - Images that will be zoomed
    rotation_range - Degree of image rotation
    width_shift_range - Image Width Change Range
    height_shift_range - Image height change range
    horizontal_flip - Rotate images horizontally
    vertical_flip - Rotate images vertically
    validation_split - Images that have been reserved for validation (0-1)

In [7]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.1,
                                    rotation_range = 20,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    horizontal_flip = True,
                                    vertical_flip = True,
                                    validation_split = 0.1)

test_datagen = ImageDataGenerator(rescale = 1./255)

## 5. Directory of training, validation and test images

Here we make the division of the image bases for training, validation and testing of the model, for that we use the [flow_from_dataframe](https://keras.io/api/preprocessing/image/#flowfromdataframe-method)

Parameters of ``flow_from_directory``:

    dataframe - Dataframe containing the images directory
    x_col - Column name containing the images directory
    y_col - Name of the column containing what we want to predict
    target_size - size of the images (remembering that it must be the same size as the input layer)
    color_mode - RGB color standard
    class_mode - binary class mode (cat/dog)
    batch_size - batch size (32)
    shuffle - Shuffle the data
    seed - optional random seed for the shuffle
    subset - Subset of data being training and validation (only used if using validation_split in ImageDataGenerator)

In [8]:
trainset_df

,File,Label
0,/home/jupyter/RealCNN/dataset/training_set/cat...,cats
1,/home/jupyter/RealCNN/dataset/training_set/cat...,cats
2,/home/jupyter/RealCNN/dataset/training_set/dog...,dogs
3,/home/jupyter/RealCNN/dataset/training_set/cat...,cats
4,/home/jupyter/RealCNN/dataset/training_set/cat...,cats
...,...,...
7675,/home/jupyter/RealCNN/dataset/training_set/dog...,dogs
7676,/home/jupyter/RealCNN/dataset/training_set/dog...,dogs
7677,/home/jupyter/RealCNN/dataset/training_set/dog...,dogs
7678,/home/jupyter/RealCNN/dataset/training_set/dog...,dogs


In [9]:

print("Preparing the training dataset ...")
training_set = train_datagen.flow_from_dataframe(
    dataframe = trainset_df,
    x_col = "File",
    y_col = "Label",
    target_size = target_size[:-1],
    color_mode = "rgb",
    class_mode = "binary",
    batch_size = batch_size,
    shuffle = True,
    seed = 2,
    subset = "training")


Preparing the training dataset ...
Found 6912 validated image filenames belonging to 2 classes.


In [10]:
def sample_from_generator(gen, nb_sample):
    from tqdm import tqdm
    cur_x, cur_y = next(gen)
    input_shape = list(cur_x.shape)[1:]
    batch_size = len(cur_x)

    X_sample = np.zeros([nb_sample] + list(input_shape))
    Y_sample = np.zeros((nb_sample,))
    
    
    X_sample[0:0 + batch_size] = cur_x
    Y_sample[0:0 + batch_size] = cur_y

    for i in tqdm(range(batch_size, nb_sample, batch_size)):
        cur_x, cur_y = next(gen)
        if len(X_sample[i:i + batch_size]) < len(cur_x):
            cur_x = cur_x[:len(X_sample[i:i + batch_size])]
            cur_y = cur_y[:len(Y_sample[i:i + batch_size])]
        if len(X_sample[i:i + batch_size]) >= len(cur_x):
            shape = cur_x.shape[0]
            X_sample[i:i + shape] = cur_x
            Y_sample[i:i + shape] = cur_y
    return X_sample, Y_sample

In [11]:
X_sample, Y_sample = sample_from_generator(training_set, batch_size*num_batches_per_epoch)
X_sample.shape

100%|██████████| 59/59 [01:33<00:00,  1.58s/it]


(7680, 128, 128, 3)

In [12]:

print("Preparing the training dataset ...")
training_set = train_datagen.flow_from_dataframe(
    dataframe = trainset_df,
    x_col = "File",
    y_col = "Label",
    target_size = target_size[:-1],
    color_mode = "rgb",
    class_mode = "binary",
    batch_size = batch_size,
    shuffle = True,
    seed = 2,
    subset = "training")

Preparing the training dataset ...
Found 6912 validated image filenames belonging to 2 classes.


In [13]:
def Alex_model(compile_model=False):
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAveragePooling2D, SpatialDropout2D
    #Instantiation
    AlexNet = Sequential()

    #1st Convolutional Layer
    AlexNet.add(Conv2D(filters=96, input_shape=target_size, kernel_size=(11,11), strides=(4,4), padding='same', activation='relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #2nd Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1,1), padding='same', activation='relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #3rd Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

    #4th Convolutional Layer
    AlexNet.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))

    #5th Convolutional Layer
    AlexNet.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same', activation='relu'))
    AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='same'))

    #Passing it to a Fully Connected layer
    AlexNet.add(Flatten())
    # 1st Fully Connected Layer
    AlexNet.add(Dense(4096, input_shape=(32,32,3,), activation='relu'))
    # Add Dropout to prevent overfitting
    AlexNet.add(Dropout(0.4))

    #2nd Fully Connected Layer
    AlexNet.add(Dense(4096, activation='relu'))
    #Add Dropout
    AlexNet.add(Dropout(0.4))

    #3rd Fully Connected Layer
    AlexNet.add(Dense(1000, activation='relu'))
    #Add Dropout
    AlexNet.add(Dropout(0.4))

    #Output Layer
    AlexNet.add(Dense(1, activation='softmax'))
    
    if compile_model:
        AlexNet.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

    return AlexNet

    

In [33]:
import psutil
print(f'current memory usage {psutil.virtual_memory().percent}%')

current memory usage 93.1%


In [18]:
# use some memory
x = np.ones((1000, 1000, 800))

In [25]:
y = np.ones((1000, 1000, 300))

In [32]:
z = np.ones((1000, 1000, 200))

In [ ]:
z1 = np.ones((1000, 1000, 200))

In [ ]:
# Callbacks
from model_trainingtime_prediction.model_level_utils_cnn import convert_cnn2d_data, TimeHistory

time_callback_data = TimeHistory()
time_callback_gen_data = TimeHistory()

CNN_data = Alex_model(compile_model=True)
CNN_gen_data = Alex_model(compile_model=True)

# Train
CNN_model = CNN_data.fit(X_sample,Y_sample, batch_size = batch_size, epochs = 50, callbacks = [time_callback_data], verbose=False)
# CNN_model = CNN_gen_data.fit(training_set, batch_size = batch_size, epochs = 3, callbacks = [time_callback_gen_data], verbose=False)


In [ ]:
batch_times = np.array(time_callback_data.batch_times) * 1000
np.median(batch_times[1:])

In [ ]:
batch_times = np.array(time_callback_gen_data.batch_times) * 1000
np.median(batch_times[1:])